In [ ]:
data = []

In [4]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Path to the directory containing your images and labels
images_dir = 'data/imagesTr'
labels_dir = 'data/labelsTr'
output_dir_images = 'data/images_png' 
output_dir_labels = 'data/labels_png'

os.makedirs(output_dir_labels, exist_ok=True)
os.makedirs(output_dir_images, exist_ok=True)

# List all the .nii.gz files in the directory
file_list = [file for file in os.listdir(images_dir) if file.endswith('.nii.gz') and file.startswith('lung')]
modified_list = [file_name[:-7] for file_name in file_list]
len_list = len(modified_list)

label_idx = 0

for idx, file in enumerate(file_list):
    # Load the label
    label_path = os.path.join(labels_dir, file)
    label = nib.load(label_path).get_fdata()

    # Check if the label contains any white (tumor) pixels
    if np.max(label) > 0:
        # Load the corresponding image
        image_path = os.path.join(images_dir, file)
        img = nib.load(image_path).get_fdata()

        # Convert slices with tumor labels to PNG
        for i in range(label.shape[2]):  # Loop through the slices
            label_slice = label[:, :, i]
            ones = np.sum(label_slice == 1.0)

            if ones > 0.015 * label_slice.shape[0]:
                slice_img = ((img[:, :, i] - np.min(img[:, :, i])) / (np.max(img[:, :, i]) - np.min(img[:, :, i]))) * 255.0
                slice_img = slice_img.astype(np.uint8)

                # Save the slice as PNG
                output_file_imgs = os.path.join(output_dir_images, ("image" + str(label_idx) + ".png"))
                output_file_labels = os.path.join(output_dir_labels, ("image" + str(label_idx) + ".png"))
                plt.imsave(output_file_imgs, slice_img)
                plt.imsave(output_file_labels, label_slice)

                label_idx = label_idx + 1

In [6]:
print('test')
a = 300

test


In [ ]:
print(a)